In [1]:
# import duckdb
# from pipelines.tasks._common import DUCKDB_FILE

# con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

In [2]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 100

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [3]:
%%sql
SELECT
    *
FROM edc_resultats
LIMIT 5;

Running query in 'duckdb:///../../database/data.duckdb'

cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date
001,00100119085,12DCLE,1161,"DICHLOROÉTHANE-1,2","Dichloroéthane-1,2",None,N,L,"<0,50",µg/L,133,<=3 µg/L,None,0.0,107-06-2,00100125759,2020,2025-02-09
001,00100119085,A2H,1832,ATRAZINE-2-HYDROXY,Atrazine-2-hydroxy,None,N,L,"<0,020",µg/L,133,"<=0,1 µg/L",None,0.0,2163-68-0,00100125759,2020,2025-02-09
001,00100119085,ACRYL,1457,ACRYLAMIDE,Acrylamide,None,N,L,"<0,10",µg/L,133,<=0.1 µg/L,None,0.0,79-06-1,00100125759,2020,2025-02-09
001,00100119085,ACTIK40,1036,ACTIVITÉ BÊTA ATTRIBUABLE AU K40,Activité bêta attribuable au K40,None,N,L,"0,034",Bq/L,9,None,None,0.034,None,00100125759,2020,2025-02-09
001,00100119085,ACTITR,2098,ACTIVITÉ TRITIUM (3H),Activité Tritium (3H),None,N,L,<8,Bq/L,9,None,<=100 Bq/L,0.0,None,00100125759,2020,2025-02-09


## Première solution

On compte par année (colonne `de_partition` qui est la colonne créée par les data engénieurs pour partitionner les données par année) le nombre de fois que chaque paramètre est apparu. Pour cela, on garde l'unicité sur ces colonnes:

- cdparametresiseeaux : Code SISE-Eaux du paramètre
- cdparametre : Code SANDRE du paramètre
- libmajparametre : Nom du paramètre en majuscule
- libminparametre : Nom du paramètre en minuscule
- casparam : Code CAS du paramètre


In [4]:
%%sql
SELECT
    de_partition , cdparametresiseeaux, cdparametre, libmajparametre, libminparametre, casparam, COUNT(*) as count
FROM edc_resultats
GROUP BY de_partition , cdparametresiseeaux, cdparametre, libmajparametre, libminparametre, casparam
ORDER BY de_partition, count DESC;

Running query in 'duckdb:///../../database/data.duckdb'

de_partition,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,count
2020,PH,1302,PH,pH,None,338110
2020,CDT25,1303,CONDUCTIVITÉ À 25°C,Conductivité à 25°C,None,263505
2020,TEAU,1301,TEMPÉRATURE DE L'EAU,Température de l'eau,None,262440
2020,ECOLI,1449,ESCHERICHIA COLI /100ML - MF,Escherichia coli /100ml - MF,None,257230
2020,STRF,6455,ENTÉROCOQUES /100ML-MS,Entérocoques /100ml-MS,None,257229
2020,CTF,1447,BACTÉRIES COLIFORMES /100ML-MS,Bactéries coliformes /100ml-MS,None,257158
2020,TURBNFU,1295,TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU,Turbidité néphélométrique NFU,None,256959
2020,ODQ,5901,ODEUR (QUALITATIF),Odeur (qualitatif),None,254069
2020,GT36_44,5441,BACT. AÉR. REVIVIFIABLES À 36°-44H,Bact. aér. revivifiables à 36°-44h,None,253182
2020,CL2LIB,1398,CHLORE LIBRE,Chlore libre,None,253105


## Deuxième solution

On va rajouter le nombre de communes distincts sur lesquels chaque paramètre est apparu.

On va devoir faire une jointure entre la table `edc_resultats` et la table `edc_prelevements` pour récupérer la colonne `inseecommuneprinc`. Or, la relation entre ces deux tables semble N-N (du moins, 1 `referenceprel` dans `edc_resultats` peut correspondre à plusieurs `referenceprel` dans `edc_prelevements`). On va donc aggréger les données de `edc_prelevements` avant la jointure et faire des comptages pour vérifier le join.


In [5]:
%%sql

select count(*) from (select * from edc_resultats)

# on compte le nombre de lignes dans la tabble ; on veut le même nombre après notre join

Running query in 'duckdb:///../../database/data.duckdb'

count_star()
64164498


In [6]:
%%sql

select count(*) from (
    with prelevements_unique as (
        select
            de_partition,
            referenceprel,
            inseecommuneprinc
        from edc_prelevements
        group by de_partition, referenceprel, inseecommuneprinc
    )
    select
        *
    from edc_resultats
    left join prelevements_unique on
        edc_resultats.referenceprel = prelevements_unique.referenceprel
        and edc_resultats.de_partition = prelevements_unique.de_partition
)

# on fait un group by et un join, on vériie que le nombre de lignes est le même

Running query in 'duckdb:///../../database/data.duckdb'

count_star()
64164498


In [7]:
%%sql


with prelevements_unique as (
    select
        de_partition,
        referenceprel,
        inseecommuneprinc
    from edc_prelevements
    group by de_partition, referenceprel, inseecommuneprinc
)
select
    edc_resultats.de_partition , cdparametresiseeaux, cdparametre, libmajparametre, libminparametre, casparam, COUNT(*) as count, COUNT(DISTINCT inseecommuneprinc) as count_communes
from edc_resultats
left join prelevements_unique on
    edc_resultats.referenceprel = prelevements_unique.referenceprel
    and edc_resultats.de_partition = prelevements_unique.de_partition
GROUP BY edc_resultats.de_partition , cdparametresiseeaux, cdparametre, libmajparametre, libminparametre, casparam
ORDER BY edc_resultats.de_partition, count DESC;

# maintenant on peut faire nos aggrégations et compter le nombre de communes en réutilisant la même jointure

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

de_partition,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,count,count_communes
2020,PH,1302,PH,pH,None,338110,34097
2020,CDT25,1303,CONDUCTIVITÉ À 25°C,Conductivité à 25°C,None,263505,34101
2020,TEAU,1301,TEMPÉRATURE DE L'EAU,Température de l'eau,None,262440,34027
2020,ECOLI,1449,ESCHERICHIA COLI /100ML - MF,Escherichia coli /100ml - MF,None,257230,34098
2020,STRF,6455,ENTÉROCOQUES /100ML-MS,Entérocoques /100ml-MS,None,257229,34097
2020,CTF,1447,BACTÉRIES COLIFORMES /100ML-MS,Bactéries coliformes /100ml-MS,None,257158,34098
2020,TURBNFU,1295,TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU,Turbidité néphélométrique NFU,None,256959,34098
2020,ODQ,5901,ODEUR (QUALITATIF),Odeur (qualitatif),None,254069,33586
2020,GT36_44,5441,BACT. AÉR. REVIVIFIABLES À 36°-44H,Bact. aér. revivifiables à 36°-44h,None,253182,33647
2020,CL2LIB,1398,CHLORE LIBRE,Chlore libre,None,253105,33181
